<a href="https://colab.research.google.com/github/ohoelske/Databases/blob/main/210116_Kursaufgabe_Woche_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Kursaufgabe Woche 2 - Asthma patients**

# Importing libraries

In [154]:
import pandas as pd
from functools import reduce
import sqlite3
from sqlite3 import Error
import csv

In [155]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [181]:
material_path = "/content/drive/MyDrive/csv_data_Oliver/asthma/"

In [185]:
# list of elements in directory
!ls "/content/drive/MyDrive/csv_data_Oliver/asthma/"

asthma_test.db	encounters.csv	     observations.xlsx	    procedures.csv
careplans.csv	imaging_studies.csv  organizations.csv	    providers.csv
conditions.csv	immunizations.csv    patients.csv	    source_asthma.db
devices.csv	medications.csv      payers.csv		    supplies.csv
disease.csv	observations.csv     payer_transitions.csv


# Path of input/output data

In [186]:
## type of patients
patient_type = "asthma"

In [187]:
# source database path
DB_FILE_PATH = f"{material_path}{patient_type}_test.db"

# folder contains csv files
csv_path = f"{material_path}"

DB_FILE_PATH, csv_path

('/content/drive/MyDrive/csv_data_Oliver/asthma/asthma_test.db',
 '/content/drive/MyDrive/csv_data_Oliver/asthma/')

#SQL queries to create tables# 

In [221]:
# dictionary key: name of table, values: sql query to create table
sql_table = {}
sql_index = {}

In [222]:
# sql query to create disease table
sql_table["disease"] = """CREATE TABLE IF NOT EXISTS disease (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)
                           
                       );"""
sql_index["disease_patient"] = """CREATE INDEX ix_disease_patient on disease(PATIENT);"""

In [223]:
#  sql query to create patients table
sql_table["patients"] = """CREATE TABLE IF NOT EXISTS patients (
                           Id STRING PRIMARY KEY,
                           BIRTHDATE DATE,
                           DEATHDATE DATE,
                           SSN STRING,
                           DRIVERS STRING,
                           PASSPORT STRING,
                           PREFIX STRING,
                           FIRST STRING,
                           LAST STRING,
                           SUFFIX STRING,
                           MAIDEN STRING,
                           MARITAL STRING,
                           RACE STRING,
                           ETHNICITY STRING,
                           GENDER STRING,
                           BIRTHPLACE STRING,
                           ADDRESS STRING,
                           CITY STRING,
                           STATE STRING,
                           COUNTRY STRING,
                           ZIP STRING,
                           LAT INTEGER,
                           LON INTEGER,
                           HEALTHCARE_EXPENSES INTEGER,
                           HEALTHCARE_COVERAGE INTEGER
                       );"""
sql_index["patients_race"] = """CREATE INDEX ix_patients_race on patients(RACE);""" 

In [224]:
# sql query to create encounters table
sql_table["encounters"] = """CREATE TABLE IF NOT EXISTS encounters (
                           Id STRING PRIMARY KEY,
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ORGANIZATIONS STRING,
                           PROVIDER STRING,
                           PAYER STRING,
                           ENCOUNTERCLASS STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_ENCOUNTER_COST INTEGER,
                           TOTAL_CLAIM_COST INTEGER,
                           PAYER_COVERAGE INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                       );"""
sql_index["encounters_patient"] = """CREATE INDEX ix_encounters_patients on encounters(PATIENT);""" 

In [225]:
# sql query to create careplans table
sql_table["careplans"] = """CREATE TABLE IF NOT EXISTS careplans (
                           Id STRING PRIMARY KEY,
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""
                       
sql_index["careplans_patient"] = """CREATE INDEX ix_careplans_patients on careplans(PATIENT);"""

In [226]:
# sql query to create conditions table
sql_table["conditions"] = """CREATE TABLE IF NOT EXISTS conditions (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""
sql_index["conditions_patient"] = """CREATE INDEX ix_conditions_patient on conditions(PATIENT);"""

In [227]:
# sql query to create medications table
sql_table["medications"] = """CREATE TABLE IF NOT EXISTS medications (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           PAYER STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           PAYER_COVERAGE INTEGER,
                           DISPENSES INTEGER,
                           TOTALCOST INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           
                       );"""
sql_index["medications_description"] = """CREATE INDEX ix_medications_description on medications(DESCRIPTION);"""

In [228]:
# # sql query to create procedures table
sql_table["procedures"]= """CREATE TABLE IF NOT EXISTS procedures (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""
sql_index["procedures_patient"] = """CREATE INDEX ix_procedures_patient on procedures(PATIENT);"""

In [229]:
# # sql query to create observations table
sql_table["observations"]= """CREATE TABLE IF NOT EXISTS observations (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           VALUE STRING,
                           UNITS STRING,
                           TYPE STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""
sql_index["observations_patient"] = """CREATE INDEX ix_observations_patient on observations(PATIENT);"""

In [230]:
# sql query to create devices table
sql_table["devices"] = """CREATE TABLE IF NOT EXISTS devices (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           UDI STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)
                           
                       );"""
sql_index["devices_patient"] = """CREATE INDEX ix_devices_patient on devices(PATIENT);"""

In [231]:
# sql query to create Imaging_studies table
sql_table["imaging_studies"] = """CREATE TABLE IF NOT EXISTS imaging_studies (
                           Id STRING PRIMARY KEY,
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           BODYSITE_CODE STRING,
                           BODYSITE_DESCRIPTION STRING,
                           MODALITY_CODE STRING,
                           MODALITY_DESCRIPTION STRING,
                           SOP_CODE STRING,
                           SOP_DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""
sql_index["imaging_studies_patient"] = """CREATE INDEX ix_imaging_studies_patient on imaging_studies(PATIENT);"""

In [232]:
# sql query to create Immunizations table
sql_table["immunizations"]= """CREATE TABLE IF NOT EXISTS immunizations(
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""
sql_index["immunizations_patient"] = """CREATE INDEX ix_immunizations_patient on immunizations(PATIENT);"""

# Import csv files into database tables

In [233]:
def connect_to_db(db_file):
    """
    Connect to an SQlite database, if db file does not exist it will be created
    :param db_file: absolute or relative path of db file
    :return: sqlite3 connection
    """
    sqlite3_conn = None

    try:
        sqlite3_conn = sqlite3.connect(db_file)
        return sqlite3_conn

    except Error as err:
        print(err)

        if sqlite3_conn is not None:
            sqlite3_conn.close()

In [234]:


def insert_values_to_table(cursor, table_name, csv_file_path):

    """
    Open a csv file, store its content in a list excluding header and insert the data from the list to db table
    :param table_name: table name in the database to insert the data into
    :param csv_file_path: path of the csv file to process
    :return: None
    """

    # Read CSV file content
    values_to_insert = open_csv_file(csv_file_path)

    # Insert to table
    if len(values_to_insert) > 0:
        column_names, column_numbers = get_column_names_from_db_table(cursor, table_name)

        values_str = '?,' * column_numbers
        # print("*****", values_str, column_names, column_numbers)
        values_str = values_str[:-1]

        sql_query = 'INSERT OR REPLACE INTO ' + table_name + '(' + column_names + ') VALUES (' + values_str + ')'


        cursor.executemany(sql_query, values_to_insert)
        # conn.commit()

        print('SQL insert process finished')
    else:
        print('Nothing to insert')

        # conn.close()

    # else:
        # print('Connection to database failed')


def open_csv_file(csv_file_path):
    """
    Open and read data from a csv file without headers (skipping the first row)
    :param csv_file_path: path of the csv file to process
    :return: a list with the csv content
    """
    with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)

        data = list()
        for row in reader:
            data.append(row)

        return data


def get_column_names_from_db_table(sql_cursor, table_name):
    """
    Scrape the column names from a database table to a list and convert to a comma separated string, count the number
    of columns in a database table
    :param sql_cursor: sqlite cursor
    :param table_name: table name to get the column names from
    :return: a comma separated string with column names, an integer with number of columns
    """

    table_column_names = 'PRAGMA table_info(' + table_name + ');'
    sql_cursor.execute(table_column_names)
    table_column_names = sql_cursor.fetchall()

    column_count = len(table_column_names)

    column_names = list()

    for name in table_column_names:
        column_names.append(name[1])

    return ', '.join(column_names), column_count


In [235]:
sql_table.keys()

dict_keys(['disease', 'patients', 'encounters', 'careplans', 'conditions', 'medications', 'procedures', 'observations', 'devices', 'imaging_studies', 'immunizations'])

In [238]:
sql_index.keys()

dict_keys(['disease_patient', 'patients_race', 'encounters_patient', 'careplans_patient', 'conditions_patient', 'medications_description', 'procedures_patient', 'observations_patient', 'devices_patient', 'imaging_studies_patient', 'immunizations_patient'])

In [239]:
# import csv into db
conn = connect_to_db(DB_FILE_PATH)
if conn is not None:
        cursor = conn.cursor()
        for name in sql_table.keys():
          csv_file = csv_path + name + ".csv"
          print(name, csv_file)
          
          cursor.execute(sql_table[name])

          insert_values_to_table(cursor, name, csv_file)
        # conn.commit()
else:
        print('Connection to database failed')

disease /content/drive/MyDrive/csv_data_Oliver/asthma/disease.csv
Nothing to insert
patients /content/drive/MyDrive/csv_data_Oliver/asthma/patients.csv
SQL insert process finished
encounters /content/drive/MyDrive/csv_data_Oliver/asthma/encounters.csv
SQL insert process finished
careplans /content/drive/MyDrive/csv_data_Oliver/asthma/careplans.csv
SQL insert process finished
conditions /content/drive/MyDrive/csv_data_Oliver/asthma/conditions.csv
SQL insert process finished
medications /content/drive/MyDrive/csv_data_Oliver/asthma/medications.csv
SQL insert process finished
procedures /content/drive/MyDrive/csv_data_Oliver/asthma/procedures.csv
SQL insert process finished
observations /content/drive/MyDrive/csv_data_Oliver/asthma/observations.csv
SQL insert process finished
devices /content/drive/MyDrive/csv_data_Oliver/asthma/devices.csv
SQL insert process finished
imaging_studies /content/drive/MyDrive/csv_data_Oliver/asthma/imaging_studies.csv
Nothing to insert
immunizations /content

In [240]:
 pd.read_csv(f'/content/drive/MyDrive/csv_data_Oliver/asthma/encounters.csv').head()

,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
0,fb4fbf02-f226-57df-4833-6ae69dbe7585,2011-06-24T09:28:17Z,2011-06-24T09:43:17Z,47f5131d-ccd1-0c75-1ecf-e86d69c0bbad,ff917dd4-ffbb-3f4c-b2df-445a7e9d29aa,16517796-2211-33db-9445-05fc9c431a3f,d47b3510-2895-3b70-9897-342d681c769d,wellness,162673000,General examination of patient (procedure),129.16,129.16,129.16,NaN,NaN
1,419132c1-42c1-d19b-5012-e156305bda0a,2014-06-27T09:28:17Z,2014-06-27T09:43:17Z,47f5131d-ccd1-0c75-1ecf-e86d69c0bbad,ff917dd4-ffbb-3f4c-b2df-445a7e9d29aa,16517796-2211-33db-9445-05fc9c431a3f,d47b3510-2895-3b70-9897-342d681c769d,wellness,162673000,General examination of patient (procedure),129.16,129.16,129.16,NaN,NaN
2,3397fd20-fbb5-b5a0-479e-a476794bc108,2017-04-14T09:28:17Z,2017-04-14T09:43:17Z,47f5131d-ccd1-0c75-1ecf-e86d69c0bbad,ff917dd4-ffbb-3f4c-b2df-445a7e9d29aa,16517796-2211-33db-9445-05fc9c431a3f,d47b3510-2895-3b70-9897-342d681c769d,wellness,162673000,General examination of patient (procedure),129.16,129.16,129.16,NaN,NaN
3,07ab566c-22bb-a641-735f-373eb17289b5,2019-04-19T09:28:17Z,2019-04-19T09:43:17Z,47f5131d-ccd1-0c75-1ecf-e86d69c0bbad,ff917dd4-ffbb-3f4c-b2df-445a7e9d29aa,16517796-2211-33db-9445-05fc9c431a3f,d47b3510-2895-3b70-9897-342d681c769d,wellness,162673000,General examination of patient (procedure),129.16,129.16,129.16,NaN,NaN
4,95e2707d-e9cc-49c8-3b8b-4d51d1e694ad,2013-07-17T21:25:09Z,2013-07-17T21:40:09Z,11df2d68-ee41-de46-08ed-1a459c75ee9a,b9a14c6c-d83f-306d-b261-778e45158c77,1d284b7d-51f7-3cef-97f7-3e7e6fab446c,047f6ec3-6215-35eb-9608-f9dda363a44c,wellness,162673000,General examination of patient (procedure),129.16,129.16,129.16,NaN,NaN


In [241]:
pd.read_csv(f'/content/drive/MyDrive/csv_data_Oliver/asthma/patients.csv').head()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,47f5131d-ccd1-0c75-1ecf-e86d69c0bbad,1977-04-08,NaN,999-52-9460,S99946427,X44075204X,Ms.,Evelin941,Marks830,NaN,NaN,S,white,nonhispanic,F,Whitman Massachusetts US,761 Durgan Alley Unit 44,Norton,Massachusetts,Bristol County,NaN,41.946921,-71.220438,894980.00,2169.76
1,11df2d68-ee41-de46-08ed-1a459c75ee9a,1979-05-02,NaN,999-87-1437,S99929162,X2897691X,Mrs.,Julia241,Vigil207,NaN,Vergara204,M,white,hispanic,F,Caguas Puerto Rico PR,330 Hudson Village,Halifax,Massachusetts,Plymouth County,NaN,41.990457,-70.889567,1051800.00,2720.60
2,416335f5-4cf8-0ce0-db50-94bffda253ea,1974-01-25,NaN,999-33-8442,S99991454,X75688756X,Mr.,Zackary401,Gorczany269,NaN,NaN,M,black,nonhispanic,M,Uxbridge Massachusetts US,968 O'Conner Promenade,Winthrop,Massachusetts,Suffolk County,NaN,42.342134,-71.034131,1240360.00,3148.08
3,32d1b4a6-fd05-9cf1-71bc-0fdb29db5f1d,1963-08-04,NaN,999-19-9921,S99935430,X17883782X,Ms.,Brittny484,Stracke611,NaN,NaN,S,white,nonhispanic,F,Nantucket Massachusetts US,857 Kris Underpass Apt 20,Duxbury,Massachusetts,Plymouth County,NaN,42.051414,-70.680573,1242900.00,3370.52
4,c49127f5-6363-6909-4d12-a548d418076f,1956-01-21,NaN,999-98-2488,S99972852,X5171852X,Mr.,José Luis472,Serrano248,NaN,NaN,S,white,hispanic,M,Santiago de los Caballeros Santiago DO,221 Roberts Union Apt 14,Lynn,Massachusetts,Essex County,1901.0,42.510079,-71.009126,197437.15,5338.73


In [242]:
pd.read_csv(f'/content/drive/MyDrive/csv_data_Oliver/asthma/disease.csv').head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION


In [243]:
pd.read_csv(f'/content/drive/MyDrive/csv_data_Oliver/asthma/medications.csv').head()

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,2014-03-08T09:27:58Z,2017-02-04T09:42:58Z,c49127f5-6363-6909-4d12-a548d418076f,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,f6fdd0f8-b874-f6ef-ea27-102653b788b2,309362,Clopidogrel 75 MG Oral Tablet,41.02,0.00,35,1435.70,NaN,NaN
1,2014-03-08T09:27:58Z,NaN,c49127f5-6363-6909-4d12-a548d418076f,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,f6fdd0f8-b874-f6ef-ea27-102653b788b2,312961,Simvastatin 20 MG Oral Tablet,23.60,0.00,82,1935.20,NaN,NaN
2,2014-03-08T09:27:58Z,NaN,c49127f5-6363-6909-4d12-a548d418076f,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,f6fdd0f8-b874-f6ef-ea27-102653b788b2,197361,Amlodipine 5 MG Oral Tablet,39.55,0.00,82,3243.10,NaN,NaN
3,2014-03-08T09:27:58Z,2017-02-04T09:42:58Z,c49127f5-6363-6909-4d12-a548d418076f,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,f6fdd0f8-b874-f6ef-ea27-102653b788b2,705129,Nitroglycerin 0.4 MG/ACTUAT Mucosal Spray,332.60,272.60,35,11641.00,NaN,NaN
4,2017-02-04T09:27:58Z,2017-02-04T09:42:58Z,c49127f5-6363-6909-4d12-a548d418076f,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,ba16d965-efab-71be-7d2c-5caedfe8c893,259255,Atorvastatin 80 MG Oral Tablet,491.49,431.49,1,491.49,NaN,NaN


In [244]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('disease',), ('encounters',), ('patients',), ('careplans',), ('conditions',), ('medications',), ('procedures',), ('observations',), ('devices',), ('imaging_studies',), ('immunizations',)]


In [245]:
cursor.execute("SELECT * FROM immunizations limit 10;")
rows = cursor.fetchall()
for row in rows:
  print(row)

('2011-06-24T09:28:17Z', '47f5131d-ccd1-0c75-1ecf-e86d69c0bbad', 'fb4fbf02-f226-57df-4833-6ae69dbe7585', 140, 'Influenza  seasonal  injectable  preservative free', 140.52)
('2014-06-27T09:28:17Z', '47f5131d-ccd1-0c75-1ecf-e86d69c0bbad', '419132c1-42c1-d19b-5012-e156305bda0a', 140, 'Influenza  seasonal  injectable  preservative free', 140.52)
('2017-04-14T09:28:17Z', '47f5131d-ccd1-0c75-1ecf-e86d69c0bbad', '3397fd20-fbb5-b5a0-479e-a476794bc108', 140, 'Influenza  seasonal  injectable  preservative free', 140.52)
('2017-04-14T09:28:17Z', '47f5131d-ccd1-0c75-1ecf-e86d69c0bbad', '3397fd20-fbb5-b5a0-479e-a476794bc108', 52, 'Hep A  adult', 140.52)
('2019-04-19T09:28:17Z', '47f5131d-ccd1-0c75-1ecf-e86d69c0bbad', '07ab566c-22bb-a641-735f-373eb17289b5', 140, 'Influenza  seasonal  injectable  preservative free', 140.52)
('2019-04-19T09:28:17Z', '47f5131d-ccd1-0c75-1ecf-e86d69c0bbad', '07ab566c-22bb-a641-735f-373eb17289b5', 113, 'Td (adult) preservative free', 140.52)
('2019-04-19T09:28:17Z', '47f

# Indexing

In [ ]:
# show created indices db_file SHOW INDEXES FROM database_name.table_name
cursor.execute("SHOW INDEX FROM encounters.csv")

Fragen:

Grafisches Dankenbankschema?

INDEXING?

Foreign Key constraints
